In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from scipy.ndimage import rotate, shift,zoom
import random
import os 
import glob
import ast

import matplotlib.pyplot as plt
import pydicom

from tqdm import tqdm

# **Chargement du modèle**

In [2]:
class ConvBlock3D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv3d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm3d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm3d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)

In [3]:
class UNet3DClassifier(nn.Module):
    def __init__(self, in_ch=1, base_ch=16):
        super().__init__()
        # Encoder
        self.enc1 = ConvBlock3D(in_ch, base_ch)        # 48³ -> 48³
        self.pool1 = nn.MaxPool3d(2)                   # -> 24³
        self.enc2 = ConvBlock3D(base_ch, base_ch*2)    # 24³ -> 24³
        self.pool2 = nn.MaxPool3d(2)                   # -> 12³

        # Bottleneck
        self.bottleneck = ConvBlock3D(base_ch*2, base_ch*4)  # 12³ -> 12³

        # Decoder
        self.up2 = nn.ConvTranspose3d(base_ch*4, base_ch*2, kernel_size=2, stride=2)  # 12³ -> 24³
        self.dec2 = ConvBlock3D(base_ch*4, base_ch*2)

        self.up1 = nn.ConvTranspose3d(base_ch*2, base_ch, kernel_size=2, stride=2)    # 24³ -> 48³
        self.dec1 = ConvBlock3D(base_ch*2, base_ch)

        # Classifier global
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool3d((1,1,1)),

            nn.Flatten(),
            nn.Linear(base_ch, 1)
        )

    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)       # 48³
        p1 = self.pool1(e1)     # 24³
        e2 = self.enc2(p1)      # 24³
        p2 = self.pool2(e2)     # 12³

        # Bottleneck
        b = self.bottleneck(p2) # 12³

        # Decoder
        d2 = self.up2(b)        # 12³ -> 24³
        d2 = torch.cat([d2, e2], dim=1)  # concat avec enc2
        d2 = self.dec2(d2)

        d1 = self.up1(d2)       # 24³ -> 48³
        d1 = torch.cat([d1, e1], dim=1)  # concat avec enc1
        d1 = self.dec1(d1)

        # Classifier
        out = self.classifier(d1)  # (B,1)
        return out.squeeze(1)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# === modèle + fonction de coût + optimiseur ===
model = UNet3DClassifier(in_ch=1, base_ch=16).to(device)
model.load_state_dict(torch.load("/kaggle/input/modele_anevrisme_03_09/pytorch/default/1/model_anvrisme_03_09.pth", map_location=device))

model = model.to(device)   # envoyer le modèle sur GPU
model.eval()

UNet3DClassifier(
  (enc1): ConvBlock3D(
    (block): Sequential(
      (0): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (1): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): ReLU(inplace=True)
      (3): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (4): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (5): ReLU(inplace=True)
    )
  )
  (pool1): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (enc2): ConvBlock3D(
    (block): Sequential(
      (0): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (1): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): ReLU(inplace=True)
      (3): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (4): InstanceNo

# **Fonctions utiles**

## **Accès à des infos**

In [5]:
def coordonnee_z(patient_path,InstanceNumber=163):
    
    dicom_files = sorted(glob.glob(patient_path+'/*.dcm'))
    slices = [pydicom.dcmread(f) for f in dicom_files]
    #tri des slices par instance number
    slices.sort(key=lambda s: int(s.InstanceNumber))

    # Trouver l’indice z dans le volume
    z_index = [i for i, s in enumerate(slices) if int(s.InstanceNumber) == InstanceNumber][0]

    return z_index

In [6]:
def get_instance_number(patient_path):

    numero_patient=os.path.basename(patient_path)
    numero_coupe=df_loc[df_loc['SeriesInstanceUID']==numero_patient]['SOPInstanceUID'].iloc[0]
    exemple_path = os.path.join(series_path,
                                  f"{numero_patient}/{numero_coupe}.dcm")
    #print('Récupération des coordonnées du voxel...')
    ds = pydicom.dcmread(exemple_path)
    
    InstanceNumber=ds.InstanceNumber

    return int(InstanceNumber)

In [7]:
def get_center(patient_path):

    numero_patient=os.path.basename(patient_path)
    numero_coupe=df_loc[df_loc['SeriesInstanceUID']==numero_patient]['SOPInstanceUID'].iloc[0]
    
    InstanceNumber=get_instance_number(patient_path)
    z=coordonnee_z(patient_path,InstanceNumber)

    coord_str = df_loc[df_loc['SOPInstanceUID'] == numero_coupe]['coordinates'].iloc[0]
    coord_dict = ast.literal_eval(coord_str)
    x = coord_dict['y']
    y = coord_dict['x'] #fait exprès

    center=np.array([x,y,z])
    return center

## **Preprocessing**

In [8]:
def resample_coordonnees(spacing, coords, target_spacing=0.4):
    x,y,z=coords
    if isinstance(target_spacing, (int, float)):
        target_spacing = (target_spacing, target_spacing, target_spacing)
    
    # coord physiques
    coords_mm = np.array([x*spacing[0], y*spacing[1], z*spacing[2]])
    # nouveaux indices
    new_voxel = coords_mm / np.array(target_spacing)
    return new_voxel

In [9]:
def preprocessing_volume_and_coords(patient_path,coords):
    "Coordonnées : np.array(x,y,z)"
    volume,spacing = dicom_to_numpy(patient_path)
    
    resample_volume = resample(volume, spacing,target_spacing=(0.4, 0.4, 0.4))
    resample_coords=resample_coordonnees(spacing, coords,target_spacing=0.4)
    
    crop_volume, crop_indices = crop(resample_volume)
    crop_coords=resample_coords - crop_indices
    
    norm_volume = normalization(crop_volume)

    return norm_volume,crop_coords

In [10]:
def dicom_to_numpy(patient_path):
    
    dicom_files = sorted(glob.glob(patient_path+'/*.dcm'))
    slices = [pydicom.dcmread(f) for f in dicom_files]

    #tri des slices par instance number
    slices.sort(key=lambda s: int(s.InstanceNumber))

    
    # On empile les pixel_array en un volume 3D NumPy (X,Y,Z)
    target_shape = slices[0].pixel_array.shape
    slices = [s for s in slices if s.pixel_array.shape == target_shape]
    volume = np.stack([s.pixel_array for s in slices], axis=-1)

    # Récupération du spacing réel
    pixel_spacing = slices[0].PixelSpacing
    dx, dy = pixel_spacing
    dz = getattr(slices[0], 'SliceThickness', 1.0)  # fallback si manquant
    
    return volume, (dx,dy,dz)

def resample(volume, spacing, target_spacing=(0.4, 0.4, 0.4)):
    zoom_factors = [s / t for s, t in zip(spacing, target_spacing)]
    new_volume = zoom(volume, zoom_factors, order=1)
    return new_volume

def crop(volume):
    """
    Coupe le volume 3D pour ne garder que la région contenant du signal.

    threshold : valeurs < threshold sont considérées comme fond/noir.
    """
    # On crée un masque des voxels non nuls
    mask = volume > (volume.max() * 0.1)
    if not mask.any():
        return volume  # rien à couper
    
    # On récupère les indices min/max pour chaque dimension
    x_min, x_max = mask.any(axis=(1,2)).nonzero()[0][[0, -1]] #axe_x
    y_min, y_max = mask.any(axis=(0,2)).nonzero()[0][[0, -1]] #axe_y
    z_min, z_max = mask.any(axis=(0,1)).nonzero()[0][[0, -1]] #axe_z
    
    # Crop
    cropped = volume[x_min:x_max+1, y_min:y_max+1, z_min:z_max+1]
    return cropped, (x_min,y_min,z_min)

def resize(volume):
    """
    Redimensionne le volume 3D à la taille target_shape par interpolation linéaire.

    """
    target_shape=(128,128,64)
    factors = [t/s for t, s in zip(target_shape, volume.shape)]
    resized_volume = zoom(volume, factors, order=1)

    return resized_volume

# Normaliser un tableau numpy 3D
def normalization(volume):
    """
    Normalise un volume entre 0 et 1 (par patient).

    """
    v_min, v_max = volume.min(), volume.max()
    if v_max > v_min:  # éviter la division par zéro
        volume = (volume - v_min) / (v_max - v_min)
    else:
        volume = np.zeros_like(volume)
    return volume

def preprocessing(patient_path):
    
    volume,spacing = dicom_to_numpy(patient_path)
    volume = resample(volume, spacing)
    volume = crop(volume)
    #volume = resize(volume)
    volume = normalization(volume)

    return volume

## **Découpe de cubes**

### **Df positif**

In [11]:
def crop_cube(volume, center, size=48):

    half = size // 2
    x, y, z = np.round(center).astype(int)

    # Calcul des bornes
    x_min, x_max = x - half, x + half
    y_min, y_max = y - half, y + half
    z_min, z_max = z - half, z + half

    # S'assurer que les bornes restent dans le volume
    x_min = max(0, x_min)
    y_min = max(0, y_min)
    z_min = max(0, z_min)
    x_max = min(volume.shape[0], x_max)
    y_max = min(volume.shape[1], y_max)
    z_max = min(volume.shape[2], z_max)

    # Extraire le cube

    cube = volume[x_min:x_max, y_min:y_max, z_min:z_max]

    return cube

In [12]:
def crop_non_overlapping(volume, center, size=48, n_close=25, n_random=25):
    cubes = {}
    X, Y, Z = volume.shape
    
 
   # --- 1. Découper volume en cubes non chevauchants
    cube_list = []
    cube_coords = []
    for i in range(0, X, size):
        for j in range(0, Y, size):
            for k in range(0, Z, size):
                cube = volume[i:i+size, j:j+size, k:k+size]

                if cube.shape == (size, size, size):  # garder seulement cubes complets
                    cube_list.append(cube)
                    cube_coords.append((i, j, k))  # coin supérieur gauche du cube
    
    # --- 2. Trouver le(s) cube(s) contenant l’anévrisme
    x, y, z = np.round(center).astype(int)
    aneurysm_idx = None
    for idx, (i, j, k) in enumerate(cube_coords):
        if (i <= x < i+size) and (j <= y < j+size) and (k <= z < k+size):
            aneurysm_idx = idx
            break
    
    if aneurysm_idx is None: 
        return {}
    
    cubes["aneurysm"] = cube_list[aneurysm_idx]

    
    # --- 3. Sélectionner les 25 cubes les plus proches (par distance euclidienne des centres)

    def cube_center(coord):
        return np.array([coord[0]+size/2, coord[1]+size/2, coord[2]+size/2])
    
    aneurysm_center = cube_center(cube_coords[aneurysm_idx])
    
    distances = []
    for idx, coord in enumerate(cube_coords):
        if idx == aneurysm_idx:
            continue
        d = np.linalg.norm(cube_center(coord) - aneurysm_center)
        distances.append((d, idx))
    
    distances.sort(key=lambda x: x[0])
    close_idxs = [idx for _, idx in distances[:n_close]]
    
    cubes["closest"] = [cube_list[idx] for idx in close_idxs]
    
    # --- 4. Sélectionner 25 cubes aléatoires parmi le reste
    remaining_idxs = [idx for _, idx in distances[n_close:]]
    random_idxs = random.sample(remaining_idxs, min(n_random, len(remaining_idxs)))
    
    cubes["random"] = [cube_list[idx] for idx in random_idxs]
    
    return cubes

### **Df négatif**

In [13]:
def get_non_overlapping_cubes(volume, cube_size=48):

    z_dim, y_dim, x_dim = volume.shape
    cubes = []

    # parcours par pas de cube_size
    for zi in range(0, z_dim - cube_size + 1, cube_size):
        for yi in range(0, y_dim - cube_size + 1, cube_size):
            for xi in range(0, x_dim - cube_size + 1, cube_size):
                cube = volume[zi:zi+cube_size, yi:yi+cube_size, xi:xi+cube_size]
                cubes.append(cube)

    return cubes

## **Visualisation d'images**

In [14]:
def show_middle_slices(volume):
    # volume shape : (X, Y, Z)
    mid_x = volume.shape[0] // 2
    mid_y = volume.shape[1] // 2

    mid_z = volume.shape[2] // 2


    fig, axes = plt.subplots(1, 3, figsize=(12, 4))

    # Coupe axiale (XY plane à profondeur z)
    axes[0].imshow(volume[:, :, mid_z].T, cmap='gray')  # transpose pour que X horizontal, Y vertical
    axes[0].set_title(f'Axiale (z={mid_z})')
    axes[0].axis('on')

    # Coupe coronale (XZ plane à coordonnée y)
    axes[1].imshow(volume[:, mid_y, :].T, cmap='gray')  # transpose pour X horizontal, Z vertical
    axes[1].set_title(f'Coronale (y={mid_y})')
    axes[1].axis('on')

    # Coupe sagittale (YZ plane à coordonnée x)
    axes[2].imshow(volume[mid_x, :, :].T, cmap='gray')  # transpose pour Y horizontal, Z vertical
    axes[2].set_title(f'Sagittale (x={mid_x})')
    axes[2].axis('on')

    plt.tight_layout()
    plt.show()

## **Manipulation de datasets**

In [15]:
def ajouter_Modality(df_main,df_info):
    df_merged = df_main.merge(
        df_info[['SeriesInstanceUID', 'Modality']],
        on='SeriesInstanceUID',
        how='left'  # conserve toutes les lignes de df_main
    )
    return df_merged

## **Prédiction**

In [16]:
def predict_cube(cube, model, threshold=0.5, device=None):
    
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.to(device)
    model.eval()
    
    # convertir en tensor et ajouter channel et batch
    cube_tensor = torch.tensor(cube, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
    
    with torch.no_grad():
        logits = model(cube_tensor)       # shape (1,)
        prob = torch.sigmoid(logits).item()
    
    label = 1 if prob > threshold else 0
    
    return prob, label

## **Detection des erreurs**

### **Sur un df positif**

In [17]:
def get_erreurs(df):
    faux_negatif=[]
    probas_faux_negatif=[]
    faux_positif=[]
    probas_faux_positif=[]

    for i in tqdm(range(len(df))):
        patient_path=os.path.join(series_path,
                                      f"{df['SeriesInstanceUID'][i]}")
        coords = get_center(patient_path)
        volume, center = preprocessing_volume_and_coords(patient_path,coords)
        cubes=crop_non_overlapping(volume, center, size=48, n_close=25, n_random=25)

        if not cubes:
            continue
            
        #cubes["aneurysm"]:

        
        prob, label=predict_cube(cubes["aneurysm"], model, threshold=0.5, device=None)
        if label==0 :
            faux_negatif.append(cubes["aneurysm"])
            probas_faux_negatif.append(prob)      
                
        for cube in cubes["closest"]:
            prob, label=predict_cube(cube, model, threshold=0.5, device=None)
            if label==1 :
                faux_positif.append(cube)
                probas_faux_positif.append(prob)
                
        for cube in cubes["random"]:
            prob,label=predict_cube(cube, model, threshold=0.5, device=None)
            if label==1 :
                faux_positif.append(cube)
                probas_faux_positif.append(prob)
    print(f'nombre de faux_negatifs : {len(faux_negatif)}')
    print(f'nombre de faux_positifs : {len(faux_positif)}')
    return faux_negatif,probas_faux_negatif,faux_positif,probas_faux_positif

### **Sur un df négatif**

In [18]:
def hard_negatives(df):
    hard_negatives=[]
    for index,row in tqdm(df.iterrows(),total=len(df)):
        patient_path=os.path.join(series_path,
                                      f"{df['SeriesInstanceUID'][index]}")
        volume=preprocessing(patient_path)
        liste_cubes=get_non_overlapping_cubes(volume, cube_size=48)
        
        for cube in liste_cubes:   
            result=predict_cube(cube, model, threshold=0.5, device=device)
            if result[1]==1 :
                hard_negatives.append(cube)

    return hard_negatives

# **Test du modèle**

In [19]:
df_loc = pd.read_csv('/kaggle/input/rsna-intracranial-aneurysm-detection/train_localizers.csv')
df_train = pd.read_csv('/kaggle/input/rsna-intracranial-aneurysm-detection/train.csv')
df_loc=ajouter_Modality(df_loc,df_train)
series_path='/kaggle/input/rsna-intracranial-aneurysm-detection/series'

df_positif=df_train[(df_train['Aneurysm Present']==1)&(df_train['Modality']=='MRI T1post')].reset_index(drop=True)
df_negatif=df_train[(df_train['Aneurysm Present']==0)&(df_train['Modality']=='MRI T1post')].reset_index(drop=True)

In [22]:
faux_negatif,probas_faux_negatif,faux_positif,probas_faux_positif=get_erreurs(df_positif[60:80].reset_index(drop=True))

100%|██████████| 17/17 [04:05<00:00, 14.43s/it]

nombre de faux_negatifs : 10
nombre de faux_positifs : 87


In [23]:
np.save("faux_negatif_4",faux_negatif)
np.save("probas_faux_negatif_4",probas_faux_negatif)
np.save("faux_positif_4",faux_positif)
np.save("probas_faux_positif_4",probas_faux_positif)

In [ ]:
for cube in faux_positif:
    show_middle_slices(cube)
print(probas_faux_positif)